In [1]:
!pip install emnist

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Flatten, Dropout
from keras.utils import to_categorical
import emnist

In [3]:
images_byclass, labels_byclass = emnist.extract_training_samples('byclass')

In [4]:
print("ByClass Split - Images:", images_byclass.shape, "Labels:", labels_byclass.shape)

ByClass Split - Images: (697932, 28, 28) Labels: (697932,)


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    images_byclass, labels_byclass, test_size=0.2, random_state=42
)

In [7]:
# (x_train, y_train), (x_test, y_test) = emnist.extract_training_samples('letters'), emnist.extract_test_samples('letters')

In [8]:
x_train.shape

(558345, 28, 28)

In [9]:
y_test.shape

(139587,)

In [10]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
x_train = x_train.reshape((558345, 28, 28, 1))
x_test = x_test.reshape((139587,28,28,1))

In [12]:
x_train.shape

(558345, 28, 28, 1)

In [13]:
x_train[0].shape

(28, 28, 1)

In [14]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [15]:
# Build the model
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3), padding="valid",activation="relu",input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides = 2 , padding='valid'))

model.add(Conv2D(32,kernel_size=(3,3), padding="valid",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides = 1 , padding='valid'))

model.add(Conv2D(32,kernel_size=(3,3), padding="valid",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides = 1 , padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(63,activation="softmax"))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 26, 26, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 11, 11, 32)        128       
 chNormalization)                                                
                                                        

In [17]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
import matplotlib.pyplot as plt
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import string

In [ ]:
# Create a list of all small letters
small_letters = list(string.ascii_lowercase)
small_letters.insert(0,'N/A')

In [ ]:
def image_preprocess(image_path):
  #convert to gray scale
  image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)/255

  image = cv2.resize(image,(28,28))

  image = image.reshape((1,28,28,1))

  return image

In [ ]:
def image_prediction(image_path):
  image = image_preprocess(image_path)

  prediction = model.predict(image)
  ans = np.where(prediction == prediction.max())

  print(f"The prediction of image is {small_letters[ans[1][0]]}")

In [ ]:
img_path = '/content/c.jpg'
image_prediction(img_path)